# IOR benchmark with pdwfs

https://ior.readthedocs.io/en/latest/userDoc/tutorial.html#getting-started-with-ior

## Benchmark script template

In [ ]:
%%bash
cat ior_script.jinja2

![IOR](https://ior.readthedocs.io/en/latest/_images/tutorial-ior-io-pattern.png)

## Load utility functions
Execute the cell below twice, one to load the python script into the cell, second to execute the cell.

In [ ]:
%load utils.py

## Benchmark

### Build IOR script

In [ ]:
# Parameters
api = "MPIIO"     # possible values: POSIX, MPIIO, HDF5 
read = "0"        # 1: perform read benchmark
numTasks="2"      # number of parallel processes
filePerProc="0"   # 1: write one file per processes
collective="1"    # 1: enable collective IO operations (MPIIO, HDF5 only)
segmentCount="1"  # see previous schematic

transferSize = ["512k", "1m", "3m", "5m", "7m", "10m","25m","35m", "50m","60m","75m","85m", "100m","115m","125m","150m","175m","200m", "225m", "250m"]

build_ior_script(api, read, numTasks, filePerProc, collective, segmentCount, transferSize)
#%cat ior_script

### Run on disk

In [ ]:
%%bash 
cd run
mpirun ior -f ../ior_script > ior_results_disk.out
# %cat ior_results_disk.out

### Run with pdwfs

In [ ]:
%%bash
cd run

# start a new Redis instance in the background
redis-cli shutdown 2> /dev/null
redis-server --daemonize yes --save "" > /dev/null

# wrap the previous command with pdwfs -p . (indicating the current directory as the intercepted directory)
pdwfs -p . -- mpirun ior -f ../ior_script > ior_results_pdwfs.out

### Parse results

In [ ]:
df_disk = parse_ior_results("run/ior_results_disk.out")
#df_disk

In [ ]:
df_pdwfs = parse_ior_results("run/ior_results_pdwfs.out")
#df_pdwfs

### Plot results

In [ ]:
%matplotlib inline
plot_results(df_disk, df_pdwfs, title="MPI-IO with collective operations")

## Example with multipe Redis instances

In [ ]:
%%bash
cd run
redis-cli -p 6379 shutdown 2> /dev/null
redis-cli -p 6380 shutdown 2> /dev/null
redis-cli -p 6381 shutdown 2> /dev/null
redis-server --daemonize yes --save "" --port 6379
redis-server --daemonize yes --save "" --port 6380
redis-server --daemonize yes --save "" --port 6381
export PDWFS_REDIS=":6379,:6380,:6381"
pdwfs -p . -- mpirun ior -a MPIIO -t 250m -b 250m -w